In [ ]:
import scipy.io as sio

data=sio.loadmat('spikewave.mat')

print(data)
spike_data=data['wave']
print(spike_data.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 

先引入我们需要用到的库，由于该数据表示5376种波的40个采样点
所以先用spike_data1保存原数据的转置，而后进行操作

In [ ]:
spike_data1 = spike_data.T
print(spike_data1.shape)
plt.figure(figsize=(50,20))
for i in range(5376):
    plt.plot(spike_data1[i])

plt.title('spike_wawes')
plt.xlabel('Sample points')
plt.ylabel('Amplitude')

问题1：把这些 spike 波形叠画在一起，观察并描述它们的可分性

肉眼可见的难以辨别，5k多条线在同一张图上实属密集，可分性不高，太密集啦

不同波的走势差异性并不是很强，难以进行划分

不妨先画几种波，浅浅分析一下

In [ ]:
plt.figure(figsize=(50,20))
for i in range(10):
    plt.plot(spike_data1[i])

plt.title('spike_wawes1')
plt.xlabel('Sample points')
plt.ylabel('Amplitude')

在这个图中可以看到，这10种波还是有一定区别的：

1、不同的波起点不同，峰值和谷值不同且峰值和谷值出现的时间各不相同。

2、这几种波在靠后的时间趋于平缓

我们再看看其他几种波

In [ ]:
plt.figure(figsize=(50,20))
for i in range(5366,5376):
    plt.plot(spike_data1[i])

plt.title('spike_wawes2')
plt.xlabel('Sample points')
plt.ylabel('Amplitude')

可以看到，这10种波的可分性更大了一些：

部分波整体都比较平缓，峰值和谷值绝对值的差相对较小，而有些波峰值和谷值绝对值的差很大。

另外，我们观察图spike_wave可以发现，还有些波在靠后的采样点峰值和谷值出现的频率较高，也就是波动比较大，这一点与我们目前分析的靠前采样点波动大而靠后采样点波动小是不同的。

至此，问题1回答完毕。

在5376种波里随机选取10条并绘制，发现每次绘制选取的10条波，差异程度均有所不同。

In [ ]:
plt.figure(figsize=(50,20))
array = np.random.randint(0,5376,10)#产生n--m之间的k个整数
for i in array:
    plt.plot(spike_data1[i])
plt.title('spike_wawes3')
plt.xlabel('Sample points')
plt.ylabel('Amplitude')

下面我们来讨论问题2：用 PCA 方法把这些 spike 分别降维到 2 维和 3 维空间，并画出来

查阅了一些参考资料：

[参考资料_1_降维——PCA](https://zhuanlan.zhihu.com/p/77151308)

[参考链接_2_python实现PCA](https://finthon.com/python-pca/)

链接1具体讲述了PCA方法是如何实现降维的，有助于我们理解PCA方法的原理，链接2则侧重于实现，通过python采用numpy和PCA的scikit-learn实现，下面主要根据链接2来实现spike_waves的降维工作。

其他参考链接：

[参考1](https://www.cnblogs.com/pinard/p/6239403.html)

[参考2](https://www.jianshu.com/p/5a6925b161bc)

参考2需要再去研读一下

In [ ]:
from sklearn.decomposition import PCA

#创建PCA对象1，降维到2维空间
pca1 = PCA(n_components = 2)

#对数据进行转换拟合
spike_waves_2d = pca1.fit_transform(spike_data1)

#打印出数据
print(spike_waves_2d)

plt.figure(figsize = (30,20))
#设定第1维做x轴，第2维做y轴
plt.scatter(spike_waves_2d[:,0],spike_waves_2d[:,1])
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib inline----解决<Figure size 640x480 with 1 Axes>
#创建PCA对象2，降维到3维空间
pca2 = PCA(n_components = 3)

#对数据进行转换拟合
spike_waves_3d = pca2.fit_transform(spike_data1)

#打印出数据
#print(spike_waves_3d)

fig = plt.figure(figsize=(10,10))
#ax = Axes3D(fig)
ax = Axes3D(fig,auto_add_to_figure=False)
fig.add_axes(ax)
#ax=fig.add_subplot(111,projection='3d')
#设定第1维做x轴，第2维做y轴,第3维做z轴
plt.scatter(spike_waves_3d[:,0],spike_waves_3d[:,1],spike_waves_3d[:,2])
#ax.scatter(spike_waves_3d[:,0],spike_waves_3d[:,1],spike_waves_3d[:,2],s=8)
#plt.show()

出现了一些问题……换个方式重来

上面这个图点点有亿点点大

太奇怪了啊啊啊啊啊

原因是因为python3.4以后把语句ax = Axes3D(fig)淘汰掉了，换成现在代码中的部分，就解决啦


[参考链接_1](https://www.jianshu.com/p/b563920fa7a8)

In [ ]:
#创建PCA对象2，降维到3维空间
pca2 = PCA(n_components = 3)

#对数据进行转换拟合
spike_waves_3d = pca2.fit_transform(spike_data1)

#打印出数据
#print(spike_waves_3d)

fig = plt.figure(figsize=(10,15))
#设定第1维做x轴，第2维做y轴,第3维做z轴
ax=fig.add_subplot(111,projection='3d')
#ax = Axes3D(fig)
ax.scatter(spike_waves_3d[:,0],spike_waves_3d[:,1],spike_waves_3d[:,2])

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

至此，问题2回答完毕。

问题3：在 2 维和 3 维空间分别采用合适的聚类方法把它们分成合理类别，这些类别就对应于临近的神经细胞放电。 

[参考资料_1_K-means算法](https://blog.csdn.net/fztsilly/article/details/113817397)

根据参考资料中的代码，将输入改为我们用PCA方法降维的二维数据并进行聚类，得到如下结果：

In [ ]:
def distance(e1, e2):
    return np.sqrt((e1[0] - e2[0]) ** 2 + (e1[1] - e2[1]) ** 2)


def means(arr):
    return np.array([np.mean([e[0] for e in arr]), np.mean([e[1] for e in arr])])


def farthest(k_arr, arr):
    f = [0, 0]
    max_d = 0
    for e in arr:
        d = 0
        for i in range(k_arr.__len__()):
            d = d + np.sqrt(distance(k_arr[i], e))
        if d > max_d:
            max_d = d
            f = e
    return f


def closest(a, arr):
    c = arr[1]
    min_d = distance(a, arr[1])
    arr = arr[1:]
    for e in arr:
        d = distance(a, e)
        if d < min_d:
            min_d = d
            c = e
    return c


if __name__ == "__main__":

    arr = spike_waves_2d
    #这里就已经实现聚类了
    m = 4   #聚类的中心点个数
    r = np.random.randint(arr.__len__() - 1)
    k_arr = np.array([arr[r]])
    cla_arr = [[]]
    for i in range(m - 1):
        k = farthest(k_arr, arr)
        k_arr = np.concatenate([k_arr, np.array([k])])
        cla_arr.append([])

    n = 20
    cla_temp = cla_arr
    for i in range(n):
        for e in arr:
            ki = 0
            min_d = distance(e, k_arr[ki])
            for j in range(1, k_arr.__len__()):
                if distance(e, k_arr[j]) < min_d:
                    min_d = distance(e, k_arr[j])
                    ki = j
            cla_temp[ki].append(e)

        for k in range(k_arr.__len__()):
            if n - 1 == i:
                break
            k_arr[k] = means(cla_temp[k])
            cla_temp[k] = []

    col = ['HotPink', 'Aqua', 'Chartreuse', 'yellow', 'LightSalmon','red']
    #col = ['HotPink', 'Aqua', 'Chartreuse', 'yellow']
    for i in range(m):
        plt.scatter(k_arr[i][0], k_arr[i][1], linewidth=10, color=col[i])
        plt.scatter([e[0] for e in cla_temp[i]], [e[1] for e in cla_temp[i]], color=col[i],s=5)
    plt.show()

继续查阅资料发现sklearn中有包装好的K-means包，以下实现方式是直接采用已封装好的KMeans来实现

[参考资料_2_K-means聚类](https://blog.51cto.com/liangdongchang/3120268)

[参考资料_3_python实现聚类分析和数据降维](https://developer.aliyun.com/article/1123835)


In [ ]:
#1、导包
from sklearn.cluster import KMeans
%matplotlib inline

#2、初始化数据
X = spike_waves_2d
plt.scatter(X[:,0],X[:,1],s=8)

#3、找到，并绘制中心点（此处我们选择的中值数量为4）
#选取4个种子
km4 = KMeans(n_clusters=4)
# 无监督学习：estimator.fit(X)
# 监督学习：estimator.fit(X,y)
#我们采用无监督学习方式，拟合的时候不需要样本标签
km4.fit(X)

#查看聚类中心坐标点
cluster_centers_ = km4.cluster_centers_
print(cluster_centers_)

#显示聚类中心
plt.scatter(X[:,0],X[:,1],s=8)
plt.scatter(cluster_centers_[:,0],cluster_centers_[:,1],s=30,alpha=0.4,c='r')

#4、用不同颜色表示不同的类别

#可以看到与上述我们没有用包，直接计算得来的分类结果差不多
y_predict2_4 = km4.predict(X)  # 预测
plt.scatter(X[:,0],X[:,1],c=y_predict2_4,s=8)  # 预测为同一簇的样本同颜色
plt.show()

#5、若分为5类，又是另一种结果
# 创建KMeans算法对象，设置聚成5类
km5 = KMeans(n_clusters=5,random_state=666) 
km5.fit(X) # 无监督学习，拟合的时候不需要样本标签
y_predict2_5 = km5.predict(X)  # 预测
plt.scatter(X[:,0],X[:,1],c=y_predict2_5,s=8)  # 预测为同一簇的样本同颜色
plt.show()

下面我们对三维数据进行聚类分析

[参考链接](https://blog.csdn.net/sgld995/article/details/108390774?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168778917216800215030551%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168778917216800215030551&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_positive~default-1-108390774-null-null.142^v88^control_2,239^v2^insert_chatgpt&utm_term=%E4%B8%89%E7%BB%B4%E6%95%B0%E6%8D%AE%E8%81%9A%E7%B1%BB&spm=1018.2226.3001.4187)

In [ ]:
import csv
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D
# 标准化数据集 

def normalize(X, axis=-1, p=2):
    lp_norm = np.atleast_1d(np.linalg.norm(X, p, axis))
    lp_norm[lp_norm == 0] = 1
    return X / np.expand_dims(lp_norm, axis)


# 计算一个样本与数据集中所有样本的欧氏距离的平方
def euclidean_distance(one_sample, X):
    one_sample = one_sample.reshape(1, -1)
    X = X.reshape(X.shape[0], -1)
    distances = np.power(np.tile(one_sample, (X.shape[0], 1)) - X, 2).sum(axis=1)
    return distances


class Kmeans():
    def __init__(self, k=4, max_iterations=500, varepsilon=0.0001):
        self.k = k
        self.max_iterations = max_iterations
        self.varepsilon = varepsilon

    # 从所有样本中随机选取self.k样本作为初始的聚类中心
    def init_random_centroids(self, X):
        n_samples, n_features = np.shape(X)
        centroids = np.zeros((self.k, n_features))
        for i in range(self.k):
            centroid = X[np.random.choice(range(n_samples))]
            centroids[i] = centroid
        return centroids

    # 返回距离该样本最近的一个中心索引[0, self.k)
    def _closest_centroid(self, sample, centroids):
        distances = euclidean_distance(sample, centroids)
        closest_i = np.argmin(distances)
        return closest_i

    # 将所有样本进行归类，归类规则就是将该样本归类到与其最近的中心
    def create_clusters(self, centroids, X):
        n_samples = np.shape(X)[0]
        clusters = [[] for _ in range(self.k)]
        for sample_i, sample in enumerate(X):
            centroid_i = self._closest_centroid(sample, centroids)
            clusters[centroid_i].append(sample_i)
        return clusters

    # 对中心进行更新
    def update_centroids(self, clusters, X):
        n_features = np.shape(X)[1]
        centroids = np.zeros((self.k, n_features))
        for i, cluster in enumerate(clusters):
            centroid = np.mean(X[cluster], axis=0)
            centroids[i] = centroid
        return centroids

    # 将所有样本进行归类，其所在的类别的索引就是其类别标签
    def get_cluster_labels(self, clusters, X):
        y_pred = np.zeros(np.shape(X)[0])
        for cluster_i, cluster in enumerate(clusters):
            for sample_i in cluster:
                y_pred[sample_i] = cluster_i
        return y_pred

    # 对整个数据集X进行Kmeans聚类，返回其聚类的标签
    def predict(self, X):
        # 从所有样本中随机选取self.k样本作为初始的聚类中心
        centroids = self.init_random_centroids(X)

        # 迭代，直到算法收敛(上一次的聚类中心和这一次的聚类中心几乎重合)或者达到最大迭代次数
        for _ in range(self.max_iterations):
            # 将所有进行归类，归类规则就是将该样本归类到与其最近的中心
            clusters = self.create_clusters(centroids, X)
            former_centroids = centroids

            # 计算新的聚类中心
            centroids = self.update_centroids(clusters, X)

            # 如果聚类中心几乎没有变化，说明算法已经收敛，退出迭代
            diff = centroids - former_centroids
            if diff.any() < self.varepsilon:
                break

        return self.get_cluster_labels(clusters, X)
# 读取数据
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
A = np.array(spike_waves_3d)
X=min_max_scaler.fit_transform(A)
num, dim = X.shape
clf = Kmeans(k=4)
y_predict3_4 = clf.predict(X)
print(y_predict3_4)
color = ['HotPink', 'Aqua', 'Chartreuse', 'yellow', 'LightSalmon','red']
ax = plt.subplot(111, projection='3d')

for p in range(0,num):
    y=y_predict3_4[p]
    ax.scatter(int(A[p, 0]), int(A[p, 1]), int(A[p, 2]), c=color[int(y)])
plt.show()


至此，问题3回答完毕

问题四：把 Spike sorting 之后的波形按照神经细胞归类，用不同颜色再次叠画在一起，观察并描述它们的波形在类内和类间的差异

我下面将按照K-means对二维数据的4聚类分类结果来绘制波形图

In [ ]:
# 获取不同类别的索引
unique_clusters = np.unique(y_predict2_4)

# 设置颜色映射，每个类别对应不同的颜色
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_clusters)))

# 绘制波形图
fig, ax = plt.subplots()
for i, y in enumerate(unique_clusters):
    mask = y_predict2_4 == y
    ax.plot(spike_data1[mask].T, color=colors[i])

ax.set_xlabel('SamplePoints')
ax.set_ylabel('Amplitude')
ax.set_title('spike_data by Cluster')

plt.show()

观察波形在类间的差异：发现不同类型的波形在时刻0-8以及8-14左右差异明显，但是到了15以后，差异变得不那么突出了

In [ ]:
# 单独绘制每个聚类簇的图形
for i, y in enumerate(unique_clusters):
    mask = y_predict2_4 == y
    cluster_data = spike_data1[mask]

    # 创建一个新的图形窗口
    fig, ax = plt.subplots()

    # 绘制每个聚类簇的数据
    for data in cluster_data:
        ax.plot(data, color=colors[i])

    ax.set_xlabel('SamplePoints')
    ax.set_ylabel('Amplitude')
    ax.set_title(f'spike_data for Cluster {y_predict2_4}')

    plt.show()

观察类内之间的差异：发现每种波形他们在0-15刻相互之间差异并不是很大，波动较小，但是15刻之后差异变得非常明显，表现为纵轴跨越度大。